In [1]:
%matplotlib inline

from matplotlib import pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.set_printoptions(edgeitems=2)
torch.manual_seed(42)

In [2]:
if torch.cuda.is_available():
    print('Using Pytorch with GPU acceleration')
else:
     print('Using CPU')

Using CPU


In [3]:
import pandas as pd

url = 'https://raw.githubusercontent.com/deep-learning-with-pytorch/dlwpt-code/master/data/p1ch4/tabular-wine/winequality-white.csv'
df = pd.read_csv(url, sep=';')

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [4]:
# prepare data - 1

# separate x and y
x = df.drop('quality', axis = 1)
x_norm = (x-x.min())/(x.max()-x.min())
y = df.pop('quality')

# obtain data tensors
x_tens = torch.from_numpy(x_norm.values).float()

# regression approach
y_tens = torch.from_numpy(y.values).float().unsqueeze(-1)

# classification approach
y_tens_c = torch.zeros(y_tens.shape[0], 10)
y_tens_c.scatter_(1, torch.from_numpy(y.values).unsqueeze(1), 1.0)

tensor([[0., 0.,  ..., 0., 0.],
        [0., 0.,  ..., 0., 0.],
        ...,
        [0., 0.,  ..., 0., 0.],
        [0., 0.,  ..., 0., 0.]])

In [5]:
# prepare data - 2

# train and validation indices
n_samples = x_tens.shape[0]
n_val = int(0.2 * n_samples)
n_test = int(0.1 * n_samples)

shuffled_indices = torch.randperm(n_samples)

trainval_indices = shuffled_indices[:-n_test]
test_indices = shuffled_indices[-n_test:]

train_indices = trainval_indices[:-n_val]
val_indices = trainval_indices[-n_val:]

# data splitting
train_x = x_tens[train_indices]
train_y = y_tens[train_indices]
train_y_c = y_tens_c[train_indices]

val_x = x_tens[val_indices]
val_y = y_tens[val_indices]
val_y_c = y_tens_c[val_indices]

test_x = x_tens[test_indices]
test_y = y_tens[test_indices]
test_y_c = y_tens_c[test_indices]

print(f' Train:\tx={train_x.shape}, y={train_y.shape}\n',
      f'Valid:\tx={val_x.shape}, y={val_y.shape}\n',
      f'Test:\tx={test_x.shape}, y={test_y.shape}')

 Train:	x=torch.Size([3430, 11]), y=torch.Size([3430, 1])
 Valid:	x=torch.Size([979, 11]), y=torch.Size([979, 1])
 Test:	x=torch.Size([489, 11]), y=torch.Size([489, 1])


In [6]:
# training loop

def training_loop(n_epochs, model, loss_fn, optimizer,
                  train_x, train_y, val_x, val_y):
    
    for epoch in range(1, n_epochs+1):
        
        train_preds = model(train_x)
        train_loss = loss_fn(train_preds, train_y)

        with torch.no_grad():
            val_preds = model(val_x)
            val_loss = loss_fn(val_preds, val_y)

        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        if epoch <= 3 or epoch % 500 == 0:
            print(f"Epoch {epoch:>4d}\tTraining loss {train_loss.item():.4f}\tValidation loss {val_loss.item():.4f}")

In [7]:
# define models

# linear model
model_1 = nn.Linear(11, 1)

# nnet for regression
model_2 = nn.Sequential(
    nn.Linear(11, 32),
    nn.ReLU(),
    nn.Linear(32, 16),
    nn.ReLU(),
    nn.Linear(16, 1)
    )

# nnet for classifiation
model_3 = nn.Sequential(
    nn.Linear(11, 32),
    nn.ReLU(),
    nn.Linear(32, 16),
    nn.ReLU(),
    nn.Linear(16, 10)
    )

# utils
metrics = dict()
error_func = nn.MSELoss()

In [8]:
# training
training_loop(
    n_epochs = 5000,
    model = model_1,
    loss_fn = nn.MSELoss(),
    optimizer = optim.Adam(model_1.parameters(), lr=1e-2),
    train_x = train_x, train_y = train_y,
    val_x = val_x, val_y = val_y,
)

Epoch    1	Training loss 33.6267	Validation loss 32.8758
Epoch    2	Training loss 33.2177	Validation loss 32.4710
Epoch    3	Training loss 32.8114	Validation loss 32.0689
Epoch  500	Training loss 0.9561	Validation loss 0.8980
Epoch 1000	Training loss 0.8381	Validation loss 0.8021
Epoch 1500	Training loss 0.7640	Validation loss 0.7416
Epoch 2000	Training loss 0.7169	Validation loss 0.6990
Epoch 2500	Training loss 0.6826	Validation loss 0.6648
Epoch 3000	Training loss 0.6565	Validation loss 0.6380
Epoch 3500	Training loss 0.6361	Validation loss 0.6172
Epoch 4000	Training loss 0.6197	Validation loss 0.6005
Epoch 4500	Training loss 0.6066	Validation loss 0.5868
Epoch 5000	Training loss 0.5966	Validation loss 0.5761


In [9]:
metrics['linear'] = error_func(model_1(test_x), test_y).item()

In [10]:
# training
training_loop(
    n_epochs = 5000,
    model = model_2,
    loss_fn = nn.MSELoss(),
    optimizer = optim.Adam(model_2.parameters(), lr=1e-2),
    train_x = train_x, train_y = train_y,
    val_x = val_x, val_y = val_y,
)

Epoch    1	Training loss 37.7217	Validation loss 36.9344
Epoch    2	Training loss 36.5683	Validation loss 35.7956
Epoch    3	Training loss 35.3890	Validation loss 34.6304
Epoch  500	Training loss 0.5994	Validation loss 0.5780
Epoch 1000	Training loss 0.5769	Validation loss 0.5525
Epoch 1500	Training loss 0.5609	Validation loss 0.5346
Epoch 2000	Training loss 0.5508	Validation loss 0.5253
Epoch 2500	Training loss 0.5458	Validation loss 0.5189
Epoch 3000	Training loss 0.5445	Validation loss 0.5194
Epoch 3500	Training loss 0.5275	Validation loss 0.5119
Epoch 4000	Training loss 0.5091	Validation loss 0.4821
Epoch 4500	Training loss 0.5038	Validation loss 0.4787
Epoch 5000	Training loss 0.4945	Validation loss 0.4832


In [11]:
metrics['nnet_reg'] = error_func(model_2(test_x), test_y).item()

In [12]:
# training
training_loop(
    n_epochs = 5000,
    model = model_3,
    loss_fn = nn.CrossEntropyLoss(),
    optimizer = optim.Adam(model_3.parameters(), lr=1e-1),
    train_x = train_x, train_y = train_y_c,
    val_x = val_x, val_y = val_y_c,
)

Epoch    1	Training loss 2.3402	Validation loss 2.3328
Epoch    2	Training loss 1.7186	Validation loss 1.6889
Epoch    3	Training loss 1.4455	Validation loss 1.3663
Epoch  500	Training loss 1.0418	Validation loss 1.0199
Epoch 1000	Training loss 1.0278	Validation loss 1.0232
Epoch 1500	Training loss 1.0301	Validation loss 1.0463
Epoch 2000	Training loss 1.0277	Validation loss 1.0593
Epoch 2500	Training loss 1.0185	Validation loss 1.0480
Epoch 3000	Training loss 1.0246	Validation loss 1.0527
Epoch 3500	Training loss 1.0132	Validation loss 1.0477
Epoch 4000	Training loss 1.0129	Validation loss 1.0521
Epoch 4500	Training loss 1.0127	Validation loss 1.0511
Epoch 5000	Training loss 1.0297	Validation loss 1.0423


In [13]:
_, preds_3 =torch.max(model_3(test_x), dim=1)
preds_3 = preds_3.unsqueeze(1)
metrics['nnet_clf'] = error_func(preds_3, test_y).item()

In [14]:
# results
print(f'Test set metrics:')
for k, v in metrics.items():
    print(f'Model: {k}\trmse: {np.sqrt(v):.4f}')

Test set metrics:
Model: linear	rmse: 0.7198
Model: nnet_reg	rmse: 0.6653
Model: nnet_clf	rmse: 0.7540
